In [1]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import onnxruntime
import numpy as np
from reid_onnx_helper import ReidHelper
import settings
import os

In [2]:
def calc_distance(feat1, feat2):
    distance = np.linalg.norm(feat1 - feat2)
    return distance

In [4]:
def make_gallery(df, query_df, number=10):
    gallery = df.sample(number)
    tmp = df.loc[(df.car == query_df.iloc[0, 1]) & (df.year == query_df.iloc[0, 2]) & (df.color == query_df.iloc[0, 6])].sample()
    gallery = pd.concat([gallery, tmp])
    
    return gallery

In [5]:
def get_rank_group(query_df, gallery, rank=1):
    file_dir = './crop/crop'
    query_feat = helper.infer(cv2.imread(file_dir + f'/{query_df.index[0]}.png'))
    distance = []
    min_distance = 0
    min_file = None
    for i in gallery.index:
        car_img = cv2.imread(file_dir + f'/{i}.png')
        feat = helper.infer(car_img)
        distance.append(calc_distance(query_feat, feat))
    gallery['distance'] = distance
    sorted_gallery = gallery.sort_values('distance')
    
    return sorted_gallery.iloc[:rank]

In [6]:
def evaluate(df, reps=100):
    cnt = 0
    for _ in range(reps):
        query_df = df.sample()
        gallery = make_gallery(df, query_df)
        rank_gallery = get_rank_group(query_df, gallery)
        result = rank_gallery.loc[(rank_gallery.car == query_df.iloc[0, 1]) & (rank_gallery.year == query_df.iloc[0, 2]) & (rank_gallery.color == query_df.iloc[0, 6])]
        if len(result) > 0:
            cnt += 1
    return cnt/reps

In [7]:
if __name__ == '__main__':
    df = pd.read_csv('./crop/crop.csv', index_col=0)
    helper = ReidHelper(settings.ReID)
    result = evaluate(df, reps=100)
    print(result)

0.82


Rank 3

vehicleid/ 3m39.9s/ 0.92

veri/ 3m39.3s/ 0.93

veriwild/ 13m2.6s/ 0.98

Rank 1

veri/ 56.6s/ 0.56

vehicleid/ 3m39.4/ 0.72

veriwild/ 12m59.6s/ 0.82